In [ ]:
#import time
#import numpy as np
#import matplotlib.pyplot as plt
#from IPython import display
Num_T = 3
Tp = np.transpose
# Exp_log is a data structure containing all data collected in a given execution 
class B_Vg_R_data:
    def __init__(self,IMag,GateV,delay):
        self.IMag = np.array(IMag) # 1Darray of Magnet Current
        self.GateV = np.array(GateV) # 1Darray of dc voltages
        self.Vxx = np.zeros((len(IMag),len(GateV),2)) # 2Darray of ac voltages readings from lockin
        self.Vxy = np.zeros((len(IMag),len(GateV),2)) # 2Darray of ac voltages readings from lockin
        self.Vth = np.zeros((len(IMag),len(GateV),2)) # 2Darray of ac voltages readings from lockin
        self.Rxx = np.zeros((len(IMag),len(GateV)))
        self.density = np.zeros((len(IMag),len(GateV)))
        self.T=np.zeros(2) # average and std 
        #self.phase = np.zeros(len(dc_voltage)) # array of phase readings from lockin
        self.delay = delay # acquisition delay 
        self.filename= ''
# input parameters:
#    k2400 is a keithley object, assumed initialized
#    srs is a srs lockin object, assumed initialized
#    dc_voltage is an array of dc voltages for the keithley
#    delay is the time to delay before acquiring data at each dc voltage
def B_Vg_R(k2400,E3640A,lock_Vxx,lock_Vxy,lock_Vth,hp6031_U,hp6031_D,IMag,GateV,delay):
    %matplotlib notebook
    
    # create new object to store data in
    data = B_Vg_R_data(IMag,GateV,delay)
    
    # make a filename to save the data, for later use
    full_filename = makefilename()
    data.filename = full_filename
    savetxtdata(full_filename+'.txt',data.IMag)
    savetxtdata(full_filename+'.txt',data.GateV)
    
    #np.savetxt(full_filename+'.txt',data.IMag,newline='\r\n')
    #np.savetxt(full_filename+'.txt',data.GateV,newline='\r\n')
    
    # define a figure with two panels to show the data being acquired
    fig = plt.figure(figsize=(8, 3)) 
    # not sure what to plot...
    fig.suptitle(full_filename)
    ax1=fig.gca(projection='3d')
    ax1.set_xlabel('Magnet Current(I)')
    ax1.set_ylabel('Gate voltage (V)')
    ax1.set_zlabel('Rxx')
    #plt.plot(data.GateV,data.ac_voltage)
    
    #data for the txt file
    #data_print = []
    #measure T from
    
    Ts=meas_T(lock_Vth,Num_T,data.delay)
    data.T=np.average(Ts),np.std(Ts)
    
    try:
        print('trying')
        # the for loop below is the main acquisition loop. the reason the for loop lives in a try-except
        # is that if the user cancels while the sweep is running, which throws an exception, the program 
        # catches it and continues to execute below the except statement. this lets the program shut down
        # nicely and save what data has already been collected.
        # main loop: iterate over dc voltages
        for i in range(len(data.IMag)):
            if one_machine:
                IsweepMag(hp6031_U,data.IMag[i])
            else:
                IsweepBoth(hp6031_U,hp6031_D,data.IMag[i])
            #print('Set Curr')
            for j in range(len(data.GateV)):
                print(i,j)
                # set voltage
                if sweep_both_gate:
                    SweepBothGate(k2400,E3640A,data.GateV[j])
                else:
                    sweep2400(k2400,data.GateV[j])

                # wait acquisition delay 
                time.sleep(data.delay)
                # read data and store it
                
                if fine_sweep_delay:
                    data.Vxx[i][j]=fine_sweep_delay_read(lock_Vxx,0)
                    #print(readsrs(lock_Vxx))
                    data.Vxy[i][j]=fine_sweep_delay_read(lock_Vxy,0)
                    data.Vth[i][j]=fine_sweep_delay_read(lock_Vth,0)
                elif auto_gain:
                    data.Vxx[i][j]=smart_auto_delay_read(lock_Vxx,0)
                    #print(readsrs(lock_Vxx))
                    data.Vxy[i][j]=smart_auto_delay_read(lock_Vxy,0)
                    data.Vth[i][j]=smart_auto_delay_read(lock_Vth,0)
                else:
                    data.Vxx[i][j]=readsrs(lock_Vxx)
                    #print(readsrs(lock_Vxx))
                    data.Vxy[i][j]=readsrs(lock_Vxy)
                    data.Vth[i][j]=readsrs(lock_Vth)                    
                #print('read')
                data.Rxx[i][j]=V_to_R(data.Vxx[i][j][0])
                savetxtdata(full_filename+'.txt',data.Vxx[i][j])
                savetxtdata(full_filename+'.txt',data.Vxy[i][j])
                savetxtdata(full_filename+'.txt',data.Vth[i][j])
                #print('savetxt')
                savetxtdata(full_filename+'.txt',[data.Rxx[i][j]])
                #print('savetxt')





#                np.savetxt(full_filename+'.txt',data.Vxx[i][j],newline='\r\n')
#                np.savetxt(full_filename+'.txt',data.Vxy[i][j],newline='\r\n')
#                np.savetxt(full_filename+'.txt',data.Vth[i][j],newline='\r\n')
#                np.savetxt(full_filename+'.txt',data.Rxx[i][j],newline='\r\n')
            #overwwrite the data at each magnetic field in case of accidents
            savedata(full_filename,data)    
            

                #print('read')
                
            # update plots
            X, Y = np.meshgrid(data.IMag, data.GateV)
            print('x,y')
            #print(X)
            #print(Y)
            #print(data.Rxx)
            
            ax1.clear()
            ax1=fig.gca(projection='3d')
            ax1.set_xlabel('Magnet Current(I)')
            ax1.set_ylabel('Gate voltage (V)')
            ax1.set_zlabel('Rxx')
            surf = ax1.plot_surface(X, Y, np.transpose(data.Rxx), cmap=cm.coolwarm, linewidth=0, antialiased=False)
            #ax1.lines[0].set_xdata(data.GateV[0:(i+1)])
            #ax1.lines[0].set_ydata(data.ac_voltage[0:(i+1)])
            #ax1.relim()
            #ax1.autoscale()
            #print('plot')
            #ax2.lines[0].set_xdata(data.dc_voltage[0:(i+1)])
            #ax2.lines[0].set_ydata(data.dc_current[0:(i+1)])
            #ax2.relim()
            #ax2.autoscale()
            #plt.tight_layout()
            ax1.zaxis.set_major_locator(LinearLocator(10))
            ax1.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

            # Add a color bar which maps values to colors.
            #fig.colorbar(surf, shrink=0.5, aspect=5)
            display.clear_output(wait=True)
            display.display(plt.gcf())
            
            #plt.show()
    except: 
        ; # do nothing if there was an exception. just proceed to what follows the try-except statement
        
    #display.clear_output(wait=True)
    
    # save data as a .pkl file for use with python
    fig.colorbar(surf, shrink=0.5, aspect=5)
    

    # zero the dc voltage
    if sweep_both_gate:
        SweepBothGate(k2400,E3640A,0)
    else:
        sweep2400(k2400,0)
    # zero the magnetic field
    if one_machine:
        IsweepMag(hp6031_U,0)
    else:
        IsweepBoth(hp6031_U,hp6031_D,0)

    return data




